CDS 486: Homework 3

Matthew Miller

02/19/19

### Importing Packages

In [114]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import geopandas
import geoplot
import geoplot.crs as gcrs
from shapely.geometry import Point, Polygon
import math
'''
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
'''
plt.style.use('dark_background')

### Import Data

In [2]:
CC_Data = pd.read_csv("data/Dataset1_combined.csv")

In [3]:
CC_Data.head()

,timestamp,latitude,longitude,altitude,speedmph,speedkmh,accuracy
0,12/22/2018 12:09:32 PM,41.603367,-86.754688,259,48.5,78.0,15.975934
1,12/22/2018 12:09:34 PM,41.603271,-86.755283,266,52.0,83.7,10.000000
2,12/22/2018 12:09:35 PM,41.603228,-86.755552,265,51.3,82.6,10.000000
3,12/22/2018 12:09:37 PM,41.603144,-86.756071,261,50.9,81.9,10.000000
4,12/22/2018 12:09:39 PM,41.603064,-86.756573,266,52.0,83.7,10.000000


In [4]:
#Tidy the timestamp data
time_list = []
for i in CC_Data["timestamp"]:
    '''This loop separates the clock time from the rest of the string'''
    start = 0 #index for slicing
    stop = 0 #index for slicing
    flag = 0
    for j in range(len(i)):
        if (i[j] == " " and flag == 0):
            start = j+1
            flag = 1
        elif (i[j] == " " and flag == 1):
            stop = j
    time_list.append(i[start:stop])

CC_Data["time"] = time_list



second_list = []
for time in CC_Data["time"]:
    '''this will create a column of all the times in seconds....this is easier to work with'''
    x = time.split(":")
    hour = int(x[0])
    minute = int(x[1])
    second = int(x[2])
    second_list.append((hour%12)*60^2 + minute*60 + second)

for i in range(len(second_list)):
    '''This changes the list so that each time is relative to the start time'''
    second_list[i] -= 574

CC_Data["seconds"] = second_list

In [5]:
#remove points where the accuracy >10
CC_Data = CC_Data[CC_Data["accuracy"] <= 10]

In [53]:
CC_Data.head()

,timestamp,latitude,longitude,altitude,speedmph,speedkmh,accuracy,time,seconds
1,12/22/2018 12:09:34 PM,41.603271,-86.755283,266,52.0,83.7,10.000000,12:09:34,2
2,12/22/2018 12:09:35 PM,41.603228,-86.755552,265,51.3,82.6,10.000000,12:09:35,3
3,12/22/2018 12:09:37 PM,41.603144,-86.756071,261,50.9,81.9,10.000000,12:09:37,5
4,12/22/2018 12:09:39 PM,41.603064,-86.756573,266,52.0,83.7,10.000000,12:09:39,7
9,12/22/2018 12:09:49 PM,41.602614,-86.759369,259,53.9,86.8,9.461421,12:09:49,17


#### Problem 1

How long is the combined journey, in miles traveled, from beginning
through end?

In [7]:
def Haversine(lat1, lon1, lat2, lon2): 
    R = 6372.8 # Earth radius in kilometers
 
    dLat = np.radians(lat2 - lat1)
    dLon = np.radians(lon2 - lon1)
    
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
 
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
 
    return R * c

In [17]:
Total_Dist = 0.0
for i in range(len(CC_Data)-1):
    Total_Dist += Haversine(lat1=CC_Data["latitude"].iloc[i], lon1=CC_Data["longitude"].iloc[i], lat2=CC_Data["latitude"].iloc[i+1], lon2=CC_Data["longitude"].iloc[i+1])

print("The total distance covered in this Cross Country was %0.3f Km or %0.3f miles" %(Total_Dist, Total_Dist*0.621371))

The total distance covered in this Cross Country was 10340.166 Km or 6425.079 miles


Plot the entire journey on a map. Please make your visual informative:
Don’t plot the data at a very high level of resolution (e.g., northern
hemisphere) such that the entire trip appears like a very short line
segment! Instead, use a bit lower resolution, so that the trip can be
followed (for example, on a map of the United States).

Which two major U.S. cities were visited on this trip? (NOTE: The
two major U.S. cities visited on this trip were those cities in which the
travelers spent more than 10 hours.) Can you identify the hotels in
which the traveler’s stayed, in each city? To do this you must understand what it might mean–from the perspective of GPS coordinates–to
spend time in a hotel. For example: What is the likely recorded speed
when someone is sleeping? There are other ways to find specific locations, but hotels are the easiest if we define “sleeping” in GPS terms.

How many total stops were made on this trip, and what are the lat/lon
coordinates of each of these stops? (NOTE: Answering this question
will require you to define a stop. That definition may or may not be
statistical in nature. If you adopt a statistical definition, then you
should provide that definition and a short discussion–a few sentences
at most–defending your choice. Also note that “stops” were not made
in the two major cities. Instead, “stops” were made in transit to the
major cities, and also between them.)

In [96]:
##speed should be zero for "n" seconds
def GetStops(n = 10, data = CC_Data):
    '''This function will gather a list of latitude/longitude pairs where each pair represents a trainstop. A trainstop is defined as stopping for at least n minutes'''
    Stops = []
    n = n*60 #time, in seconds of how long a stop is
    t = 0 #time since the last stop started
    for i in range(0,len(data)-1):
        if ((all(data["speedmph"].iloc[i:i+10]) == False) and (all(data["speedmph"].iloc[i-10:i-1]) == True)):
        #if (all points from the current point to the next 10 values are 0) and (the last 10 values before i are not 0)
            t = data["seconds"].iloc[i]
            Stops.append([data["latitude"].iloc[i],data["longitude"].iloc[i]])
            
    return Stops

In [113]:
range(CC_Data["speedmph"][18:24])

TypeError: 'Series' object cannot be interpreted as an integer

In [108]:
##speed should be zero for "n" seconds
def GetStops(n = 10, data = CC_Data):
    '''This function will gather a list of latitude/longitude pairs where each pair represents a trainstop. A new trainstop is defined as stopping for at least n minutes '''
    Stops = []
    n = n*60 #time, in seconds of how long a stop is
    t = 0 #time the last stop started
    first = 0 #checks for 1st stop
    
    for i in range(0,len(data)-1):
        if ((all(data["speedmph"].iloc[i:i+5]) == False) and (first == 0):
        elif ((all(data["speedmph"].iloc[i:i+5]) == False) and ((data["seconds"].iloc[i]-t) >= n) and (all(data["speedmph"].iloc[i-5:i]) == True)):
        #if (all points from the current point to the next 5 values are 0) and (it has been at least n seconds since the last stop)and (the last 5 values before i are not 0)
            t = data["seconds"].iloc[i]
            Stops.append([data["latitude"].iloc[i],data["longitude"].iloc[i]])
            
    return Stops

In [109]:
S = GetStops(n = 0.5)

In [110]:
S

[[41.596109000000006, -86.924899],
 [41.598246, -86.954492],
 [41.609596999999994, -87.033525],
 [41.604532, -87.25389],
 [41.604613, -87.260003],
 [41.604617, -87.26029],
 [41.605031, -87.290666],
 [41.605203, -87.30216300000001],
 [41.605328, -87.31101899999999],
 [41.605457, -87.320415],
 [41.605548, -87.327106],
 [41.605965999999995, -87.337026],
 [41.607546, -87.34510999999999],
 [41.618203, -87.375355],
 [41.620740000000005, -87.38250699999999],
 [41.624825, -87.393976],
 [44.789784000000004, -92.905244],
 [44.809761, -92.929798],
 [44.818247, -92.94385],
 [44.972744, -93.194639],
 [41.671624, -122.079128]]

In [111]:
len(S)

21

Plot the stop locations on a map.

Report the mean speed between stops, and the standard deviation of
the speed between stops.

Identify the U.S. states that were “visited” by the travelers (NOTE:
We will adopt a loose definition of “visited”: any state traversed by the
journey is considered “visited” for purposes of this question.)

There were several natural landscape and man-made landmarks visited
on this trip (a landmark being defined loosely as a tourist destination).
Name three of them. (NOTE: To assist you with this task, it will
be helpful to also plot the locations of some landmarks on a map,
and see whether the traveler’s journey “intersected with” or otherwise
spent significant time near one of these landmarks. Once you identify
the two major cities visited on the journey, you will have sufficient
information to identify at least three landmarks. How? By Googling
the names of the two cities and making a list of some of the noteworthy
landmarks in each, and then, obtaining the GPS coordinates of these
landmarks!) Can you name additional landmarks, particularly outside
densely populated areas, that were “visited” in transit? One homework
extra credit point will be awarded for every five additional landmarks
identified, up to +3 homework extra credit points. (As an aside, and
FYI only, natural landscape landmarks–of which there are several on
this journey–are best “visited” at low speeds: If not, then it becomes
hard to photograph them!)

Create a graphically pleasant Powerpoint presentation of your above
analysis of the combined journey, using the template format outlined
in Homework 2. In this case, don’t consider the journey as a problem
to be solved. Instead, consider the journey as a product to be sold. That
is precisely why you were just asked to identify states and landmarks!
Your Powerpoint presentation should reflect this “sales” viewpoint.

Create a separate document (either Powerpoint or word-processed text
document) that is a technical appendix and that includes any and all
python code that you write.